### Importing dependencies

In [1]:
import tensorflow as tf
from tensorflow.contrib.eager.python import tfe
tfe.enable_eager_execution()
from tensorflow import keras
# from tensorflow import keras.preprocessing.imageImageDataGenerator 
# import tf.contrib.keras.preprocessing.image.ImageDataGenerator as ImageDataGenerator
# # keras.preprocessing.image.ImageDataGenerator
# import keras
# from keras import layers
# from keras.preprocessing import image # keras.preprocessing.image
# from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import ResNet50
# from keras.applications.resnet50 import preprocess_input
# from keras.callbacks import ModelCheckpoint
# from keras.models import load_model #  keras.models.load_model

import matplotlib.pyplot as plt
import os, shutil
from livelossplot import PlotLossesKeras
import numpy as np

tf.__version__

Using TensorFlow backend.


'1.8.0'

In [2]:
# from tensorflow.python.client import device_lib

# device_lib.list_local_devices()

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3' 

In [4]:
traindir ='/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/train'
valdir = '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/validation'

batch_size = 32 #32
img_size = 224
epochs = 3

### Create Keras data generators 

In [5]:
#todo: figure out how to add preprocessing.

In [6]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
#     preprocessing_function=keras.applications.resnet50.preprocess_input,
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = keras.preprocessing.image.ImageDataGenerator()
#     preprocessing_function=keras.applications.resnet50.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    traindir,
    batch_size=batch_size,
    class_mode='binary',
    target_size=(img_size,img_size))

validation_generator = validation_datagen.flow_from_directory(
    valdir,
    shuffle=False,
    class_mode='binary',
    target_size=(img_size,img_size))


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Import pre-trained ResNet50

Time to load the pre-trained ResNet model. We'll freeze all layers, so there's no backpropagation of gradients through the ResNet50 layers (this saves us A LOT of time and resources)

In [7]:
keras.backend.set_learning_phase(0) # see https://github.com/keras-team/keras/issues/9214  and https://github.com/keras-team/keras/pull/9965

conv_base = keras.applications.ResNet50(
    include_top=False,
    weights='imagenet')

# freeze layers in resnet50
for layer in conv_base.layers:
    layer.trainable = False

### Stack trainable, fully-connected (dense) layers on top of it

These are the layers, which will have randomly initiated weights: these are the ones we'll actually train.

In [8]:
model = conv_base.output
keras.backend.set_learning_phase(1)
model = keras.layers.GlobalAveragePooling2D()(model)

# add fc layers
model = keras.layers.Dense(128)(model) 
predictions = keras.layers.Dense(1, activation='sigmoid')(model)

### Prepare model checkpointing

In [9]:
model = keras.Model(conv_base.input, predictions)

In [10]:
model_checkpoint_dir = "./output"

if not os.path.exists(model_checkpoint_dir):
    os.mkdir(model_checkpoint_dir)
    print('Created directory: ', model_checkpoint_dir)

model_checkpoint_dir = model_checkpoint_dir + "/best_model.hdf5"


Keras API allows us to easily save the model architecture at the stage it reaches the best validation accuracy (or any metric we want to optimize for). The saving is done through ModelCheckpoint callback.

In [11]:
checkpoint_save_best = keras.callbacks.ModelCheckpoint(model_checkpoint_dir, monitor='val_acc',
verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)

In [12]:
len(model.trainable_weights)

4

### Compile and train the model (A)

If eager mode wasn't enabled, we could compile train the model using the standard Keras API, like so:

In [13]:
# optimizer = keras.optimizers.RMSprop()

# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

But since we want to see how eager execution works, we'll do it differently.

Note that compiling a tf.keras model while tfe.enable_eager_execution() results in: 
"ValueError: Only TF native optimizers are supported in Eager mode."


In [14]:
# %%time

# history = model.fit_generator(generator=train_generator,
#                     epochs=epochs,
#                     validation_data=validation_generator, 
#                     steps_per_epoch = train_generator.n // batch_size,
#                     validation_steps = validation_generator.n // batch_size,
#                     initial_epoch=0,
#                     callbacks=[checkpoint_save_best,])
#                     #          plot_losses])    

### Train the model (B)

In [15]:
# tensorflow eager execution

In [18]:
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# optimizer = tf.keras.optimizers.RMSprop()
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)


In [21]:
def loss(model, inputs, labels):
    prediction = model(inputs)
    
    return tf.losses.sigmoid_cross_entropy(logits=prediction, multi_class_labels=tf.expand_dims(labels, dim=1))

In [22]:
def grad(model, inputs, targets):

    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, model.variables)

Let's print the loss on a single batch to see if it works so far

In [23]:
#test forward prop
with tf.device("/gpu:0"):
    x, y = iter(train_generator).next()
    print("Initial loss: {:.3f}".format(loss(model, x, y)))

Initial loss: 0.661


In [24]:
#there's something wrong with the below- it's suspiciously slow, compared to the pure keras version...

In [ ]:
with tf.device("/gpu:0"):

    # for epoch in range(epochs):
    for (batch, (inputs, labels)) in enumerate(train_generator):

        grads = grad(model, inputs, labels)

        optimizer.apply_gradients(zip(grads, model.variables),
                                    global_step=tf.train.get_or_create_global_step())
#         if batch % 200 == 0:
#         print("Loss at step {:04d}: {:.3f}".format(batch, loss(model, x, y)))
        print("Loss at step {:04d}: {:.3f}".format(batch, loss(model, x, y)))
        
print("Final loss: {:.3f}".format(loss(model, x, y)))

Loss at step 0000: 0.660
Loss at step 0001: 0.661
Loss at step 0002: 0.661
Loss at step 0003: 0.660
Loss at step 0004: 0.661
Loss at step 0005: 0.660
Loss at step 0006: 0.660
Loss at step 0007: 0.659
Loss at step 0008: 0.659
Loss at step 0009: 0.659
Loss at step 0010: 0.657
Loss at step 0011: 0.658
Loss at step 0012: 0.657
Loss at step 0013: 0.657
Loss at step 0014: 0.657
Loss at step 0015: 0.657
Loss at step 0016: 0.656
Loss at step 0017: 0.655
Loss at step 0018: 0.654
Loss at step 0019: 0.652
Loss at step 0020: 0.652
Loss at step 0021: 0.652
Loss at step 0022: 0.651
Loss at step 0023: 0.650
Loss at step 0024: 0.648
Loss at step 0025: 0.648
Loss at step 0026: 0.648
Loss at step 0027: 0.645
Loss at step 0028: 0.643
Loss at step 0029: 0.641
Loss at step 0030: 0.640
Loss at step 0031: 0.637
Loss at step 0032: 0.637
Loss at step 0033: 0.632
Loss at step 0034: 0.633
Loss at step 0035: 0.627
Loss at step 0036: 0.624
Loss at step 0037: 0.625
Loss at step 0038: 0.618
Loss at step 0039: 0.614


Loss at step 0328: 0.650
Loss at step 0329: 0.672
Loss at step 0330: 0.667
Loss at step 0331: 0.663
Loss at step 0332: 0.671
Loss at step 0333: 0.655
Loss at step 0334: 0.655
Loss at step 0335: 0.664
Loss at step 0336: 0.668
Loss at step 0337: 0.664
Loss at step 0338: 0.665
Loss at step 0339: 0.656
Loss at step 0340: 0.640
Loss at step 0341: 0.683
Loss at step 0342: 0.667
Loss at step 0343: 0.653
Loss at step 0344: 0.657
Loss at step 0345: 0.646
Loss at step 0346: 0.638
Loss at step 0347: 0.642
Loss at step 0348: 0.655
Loss at step 0349: 0.616
Loss at step 0350: 0.599
Loss at step 0351: 0.615
Loss at step 0352: 0.690
Loss at step 0353: 0.660
Loss at step 0354: 0.658
Loss at step 0355: 0.634
Loss at step 0356: 0.639
Loss at step 0357: 0.637
Loss at step 0358: 0.656
Loss at step 0359: 0.655
Loss at step 0360: 0.655
Loss at step 0361: 0.649
Loss at step 0362: 0.638
Loss at step 0363: 0.636
Loss at step 0364: 0.625
Loss at step 0365: 0.608
Loss at step 0366: 0.599
Loss at step 0367: 0.621


Loss at step 0656: 0.667
Loss at step 0657: 0.662
Loss at step 0658: 0.681
Loss at step 0659: 0.642
Loss at step 0660: 0.645
Loss at step 0661: 0.647
Loss at step 0662: 0.658
Loss at step 0663: 0.659
Loss at step 0664: 0.645
Loss at step 0665: 0.655
Loss at step 0666: 0.642
Loss at step 0667: 0.642
Loss at step 0668: 0.642
Loss at step 0669: 0.622
Loss at step 0670: 0.656
Loss at step 0671: 0.640
Loss at step 0672: 0.644
Loss at step 0673: 0.632
Loss at step 0674: 0.632
Loss at step 0675: 0.628
Loss at step 0676: 0.633
Loss at step 0677: 0.636
Loss at step 0678: 0.628
Loss at step 0679: 0.634
Loss at step 0680: 0.628
Loss at step 0681: 0.641
Loss at step 0682: 0.644
Loss at step 0683: 0.643
Loss at step 0684: 0.624
Loss at step 0685: 0.622
Loss at step 0686: 0.619
Loss at step 0687: 0.620
Loss at step 0688: 0.620
Loss at step 0689: 0.637
Loss at step 0690: 0.645
Loss at step 0691: 0.648
Loss at step 0692: 0.647
Loss at step 0693: 0.649
Loss at step 0694: 0.636
Loss at step 0695: 0.643


KeyboardInterrupt: 

In [ ]:
stop

### Load the best model and evaluate it on a sample image 

In [ ]:
keras.backend.clear_session()

In [ ]:
loaded_model = keras.models.load_model(model_checkpoint_dir, compile=False) #compile causes issues if not turned off - we are not training the model so we don't need to compile the model #the workaround to this user warning error might be to set all weights to not-trainable explicitly (in a loop) https://stackoverflow.com/questions/49195189/error-loading-the-saved-optimizer-keras-python-raspberry
print("Model loaded successfully!")

In [ ]:
test_cat = "/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/test/cats/cat.1505.jpg"

In [ ]:
img = image.load_img(test_cat, target_size=(img_size, img_size))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
preds = loaded_model.predict(x)
preds
# pred_class = loaded_model.predict_classes(x)
# pred_class = loaded_model.probas_to_classes(x)

if preds[0][0] < 0.5:
    print("IT'S A CAT! ({:.2})".format(preds[0][0]))
else:
    print("IT'S A DOG! ({:.2})".format(preds[0][0]))

In [ ]:
img = img.resize((128,128))
plt.imshow(img)
plt.show()